In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('HandsOnPertemuan3').getOrCreate()

data = [
    ('James', 'Sales', 3000),
    ('Michael', 'Sales', 4600),
    ('Robert', 'Sales', 4100),
    ('Maria', 'Finance', 3000)
]
columns = ['EmployeeName', 'Department', 'Salary']

df = spark.createDataFrame(data, schema=columns)
df.show()
df.printSchema()
df.describe().show()


25/09/10 21:48:02 WARN Utils: Your hostname, jaki-ThinkPad-T490 resolves to a loopback address: 127.0.1.1; using 192.168.0.115 instead (on interface wlp0s20f3)
25/09/10 21:48:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/09/10 21:48:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/10 21:48:03 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
                                                                                

+------------+----------+------+
|EmployeeName|Department|Salary|
+------------+----------+------+
|       James|     Sales|  3000|
|     Michael|     Sales|  4600|
|      Robert|     Sales|  4100|
|       Maria|   Finance|  3000|
+------------+----------+------+

root
 |-- EmployeeName: string (nullable = true)
 |-- Department: string (nullable = true)
 |-- Salary: long (nullable = true)



[Stage 3:>                                                          (0 + 8) / 8]

+-------+------------+----------+----------------+
|summary|EmployeeName|Department|          Salary|
+-------+------------+----------+----------------+
|  count|           4|         4|               4|
|   mean|        null|      null|          3675.0|
| stddev|        null|      null|805.708797684788|
|    min|       James|   Finance|            3000|
|    max|      Robert|     Sales|            4600|
+-------+------------+----------+----------------+



In [2]:
from pyspark.sql import functions as F

df.select('EmployeeName', 'Salary').show()

df.filter(df['Salary'] > 3000).show()

df.groupBy('Department').agg(F.avg('Salary').alias('AvgSalary')).show()

df.groupBy('Department').agg(
    F.sum('Salary').alias('TotalSalary'),
    F.max('Salary').alias('MaxSalary')
).show()


+------------+------+
|EmployeeName|Salary|
+------------+------+
|       James|  3000|
|     Michael|  4600|
|      Robert|  4100|
|       Maria|  3000|
+------------+------+

+------------+----------+------+
|EmployeeName|Department|Salary|
+------------+----------+------+
|     Michael|     Sales|  4600|
|      Robert|     Sales|  4100|
+------------+----------+------+



+----------+---------+
|Department|AvgSalary|
+----------+---------+
|     Sales|   3900.0|
|   Finance|   3000.0|
+----------+---------+

+----------+-----------+---------+
|Department|TotalSalary|MaxSalary|
+----------+-----------+---------+
|     Sales|      11700|     4600|
|   Finance|       3000|     3000|
+----------+-----------+---------+



In [3]:
df_bonus = df.withColumn('SalaryBonus', df['Salary'] * 0.1)
df_bonus.show()

df_total = df_bonus.withColumn('TotalCompensation', df_bonus['Salary'] + df_bonus['SalaryBonus'])
df_total.show()


+------------+----------+------+-----------+
|EmployeeName|Department|Salary|SalaryBonus|
+------------+----------+------+-----------+
|       James|     Sales|  3000|      300.0|
|     Michael|     Sales|  4600|      460.0|
|      Robert|     Sales|  4100|      410.0|
|       Maria|   Finance|  3000|      300.0|
+------------+----------+------+-----------+

+------------+----------+------+-----------+-----------------+
|EmployeeName|Department|Salary|SalaryBonus|TotalCompensation|
+------------+----------+------+-----------+-----------------+
|       James|     Sales|  3000|      300.0|           3300.0|
|     Michael|     Sales|  4600|      460.0|           5060.0|
|      Robert|     Sales|  4100|      410.0|           4510.0|
|       Maria|   Finance|  3000|      300.0|           3300.0|
+------------+----------+------+-----------+-----------------+



In [4]:
from pyspark.sql.window import Window

windowSpec = Window.partitionBy('Department').orderBy('Salary')

df.withColumn('Rank', F.rank().over(windowSpec)).show()

df.withColumn('RunningTotal', F.sum('Salary').over(windowSpec.rowsBetween(Window.unboundedPreceding, Window.currentRow))).show()


+------------+----------+------+----+
|EmployeeName|Department|Salary|Rank|
+------------+----------+------+----+
|       James|     Sales|  3000|   1|
|      Robert|     Sales|  4100|   2|
|     Michael|     Sales|  4600|   3|
|       Maria|   Finance|  3000|   1|
+------------+----------+------+----+

+------------+----------+------+------------+
|EmployeeName|Department|Salary|RunningTotal|
+------------+----------+------+------------+
|       James|     Sales|  3000|        3000|
|      Robert|     Sales|  4100|        7100|
|     Michael|     Sales|  4600|       11700|
|       Maria|   Finance|  3000|        3000|
+------------+----------+------+------------+



In [6]:
import kagglehub
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, max, sum

# Download dataset dari Kaggle
path = kagglehub.dataset_download("ruchi798/data-science-job-salaries")
print("Dataset downloaded to:", path)

# Inisialisasi SparkSession
spark = SparkSession.builder.appName("Tugas5_DS_Salaries").getOrCreate()

# Load file CSV
df = spark.read.option("header", True).option("inferSchema", True).csv(f"{path}/ds_salaries.csv")

# Tampilkan beberapa baris awal
print("📊 Contoh data:")
df.show(5)

# Tampilkan skema kolom
print("🧾 Struktur data:")
df.printSchema()

# 💡 Insight 1: Rata-rata gaji per experience level
print("💡 Rata-rata gaji berdasarkan pengalaman:")
df.groupBy("experience_level").agg(avg("salary_in_usd").alias("avg_salary")).show()

# 💡 Insight 2: 10 Pekerjaan dengan gaji tertinggi
print("💡 10 Pekerjaan dengan gaji tertinggi:")
df.groupBy("job_title").agg(max("salary_in_usd").alias("max_salary")) \
  .orderBy("max_salary", ascending=False).show(10)

# 💡 Insight 3: Total salary berdasarkan lokasi perusahaan
print("💡 Total gaji per lokasi perusahaan:")
df.groupBy("company_location").agg(sum("salary_in_usd").alias("total_salary")).show()

# 💡 Insight 4: Data gaji lebih dari $200k
print("💡 Daftar pekerjaan dengan gaji > $200.000:")
df.filter(df["salary_in_usd"] > 200000).select("job_title", "employee_residence", "salary_in_usd").show()


/home/jaki/anaconda3/envs/jaki/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 7.37k/7.37k [00:00<00:00, 3.26MB/s]

Extracting model files...
Dataset downloaded to: /home/jaki/.cache/kagglehub/datasets/ruchi798/data-science-job-salaries/versions/1


📊 Contoh data:
+---+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+
|_c0|work_year|experience_level|employment_type|           job_title|salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+---+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+
|  0|     2020|              MI|             FT|      Data Scientist| 70000|            EUR|        79833|                DE|           0|              DE|           L|
|  1|     2020|              SE|             FT|Machine Learning ...|260000|            USD|       260000|                JP|           0|              JP|           S|
|  2|     2020|              SE|             FT|   Big Data Engineer| 85000|            GBP|       109024|                GB|          50|  

25/09/10 21:55:25 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , work_year, experience_level, employment_type, job_title, salary, salary_currency, salary_in_usd, employee_residence, remote_ratio, company_location, company_size
 Schema: _c0, work_year, experience_level, employment_type, job_title, salary, salary_currency, salary_in_usd, employee_residence, remote_ratio, company_location, company_size
Expected: _c0 but found: 
CSV file: file:///home/jaki/.cache/kagglehub/datasets/ruchi798/data-science-job-salaries/versions/1/ds_salaries.csv


+----------------+------------------+
|experience_level|        avg_salary|
+----------------+------------------+
|              EX|199392.03846153847|
|              MI| 87996.05633802817|
|              EN|61643.318181818184|
|              SE|138617.29285714286|
+----------------+------------------+

💡 10 Pekerjaan dengan gaji tertinggi:
+--------------------+----------+
|           job_title|max_salary|
+--------------------+----------+
|Principal Data En...|    600000|
|Financial Data An...|    450000|
|  Research Scientist|    450000|
|Applied Machine L...|    423000|
|Principal Data Sc...|    416000|
|      Data Scientist|    412000|
| Data Analytics Lead|    405000|
|Applied Data Scie...|    380000|
|Director of Data ...|    325000|
|       Data Engineer|    324000|
+--------------------+----------+
only showing top 10 rows

💡 Total gaji per lokasi perusahaan:
+----------------+------------+
|company_location|total_salary|
+----------------+------------+
|              DZ|     